In [1]:
import pickle

In [2]:
with open("data/sample_000000.pkl", "rb") as f:
    sample = pickle.load(f)

In [4]:
points = sample['points']
print("Points shape:", points.shape)
print("Frame ID:", sample['frame_id'])
print("Sample keys:", sample.keys())
print(sample['points'])

Points shape: (16384, 4)
Frame ID: 000000
Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])
[[15.322624   -0.9945517  -1.3005242   0.37      ]
 [16.28796     5.4508047  -0.45010167  0.32      ]
 [11.225035    7.8189     -1.5432059   0.21      ]
 ...
 [10.327182    5.269324   -1.5712075   0.4       ]
 [13.737979   -5.4365177  -1.3347485   0.22      ]
 [12.59426    10.674768    0.26653486  0.27      ]]
